# Iteration 1: treatment - stage 2

## Plain English Summary

We increased the scope of the model by including a treatment process for unplanned (emergency) arrivals to the model.  Like arrrivals each type of patient has their own treatment distribution for sampling.  The treatment process also introduces a shared resource into the model - critical care beds - and a First come first served queue.

The input command is explicit that the code should be *modified* and that the full code listing should be produced.




## Notebook imports

In [15]:
from IPython.display import Markdown, display

import numpy as np
import math

## LLM Prompt

In [12]:
display(Markdown("../01_perplexity/inputs/iteration_1.md"))

### Main command

Modify the model to add in an unplanned admissions treatment process.

Show the full code listing.  Do not hide any functions or classes.

### General model logic

The model includes a resource: critical care beds.

After patients have arrived they follow the unplanned admissions process. In the unplanned admissions process patients request and wait for a critical care bed resource to become available in a FIFO queue, and undergo treatment.  Following treatment the patient is recorded as "discharged", but the bed is not released until intensive cleaning has taken place.  

The model should print out useful information after each event.

### Simulation inputs

the length of stay (measured in hours) for treatment of patients in the CCU follow source specific distributions

Accident and Emergency: Lognormal: mean 128.79 and standard deviation 267.51
the Wards = Lognormal: mean 177.89 and standard deviation 276.54
Emergency surgery = Lognormal: mean 140.15 and standard deviation 218.02
other hospitals = Lognormal: mean 212.86 and standard deviation 457.67
the X-Ray department = Lognormal: mean 87.53 and standard deviation 108.67

The mean and standard deviation of the Lognormal distributions must be converted to the mean and standard deviation of the underlying normal distribution.

There are 24 critical care beds in the model.  These are shared across all types of patient.

Intensive cleaning takes a fixed amount of time: 5 hours.




### Date and time of input to LLM

> 1307 30th May 2024

## Observations on the LLM Output

* The LoS parameters are hardcoded and converted to the mean and standard deviation of the lognormal to the underlying normal as follows (including a redundant step in the conversion function):

```python
    zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
    sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
    mu_norm = zeta
```
* Treatment times are sampled from the `numpy.random.lognormal` distribution using the converted parameters. A 'source' parameter is added to the unplanned_admissions process to determine which los parameters to use.
* from scipy.stats import lognorm is not used in the model 
* The model included a new `simpy` resource `critical_care_beds`. Hard coded to 24.
* The model code provided runs without raise any runtime errors

### Testing LLM code snippets 

* Test of the lognormal sampling.  The code below is our own verified code. The source is given in the docstring.

In [13]:
class Lognormal:
    """
    Encapsulates a lognormal distirbution

    Created by tester:

    Source: https://tommonks.github.io/treatment-centre-sim/content/02_model_code/04_model.html
    """
    def __init__(self, mean, stdev, random_seed=None):
        """
        Params:
        -------
        mean: float
            mean of the lognormal distribution
            
        stdev: float
            standard dev of the lognormal distribution
            
        random_seed: int, optional (default=None)
            Random seed to control sampling
        """
        self.rng = np.random.default_rng(seed=random_seed)
        mu, sigma = self.normal_moments_from_lognormal(mean, stdev**2)
        self.mu = mu
        self.sigma = sigma
        
    def normal_moments_from_lognormal(self, m, v):
        '''
        Returns mu and sigma of normal distribution
        underlying a lognormal with mean m and variance v
        source: https://blogs.sas.com/content/iml/2014/06/04/simulate-lognormal
        -data-with-specified-mean-and-variance.html

        Params:
        -------
        m: float
            mean of lognormal distribution
        v: float
            variance of lognormal distribution
                
        Returns:
        -------
        (float, float)
        '''
        phi = math.sqrt(v + m**2)
        mu = math.log(m**2/phi)
        sigma = math.sqrt(math.log(phi**2/m**2))
        return mu, sigma
        
    def sample(self):
        """
        Sample from the normal distribution
        """
        return self.rng.lognormal(self.mu, self.sigma)

In [16]:
def llm_lognormal(mu, sigma):
    """Convert lognormal parameters to underlying normal distribution"""
    zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
    sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
    mu_norm = zeta
    return mu_norm, sigma_norm

### Running the tests

We will use actual data from the case study.

In [17]:
mean, std = 128.79, 267.51
expected_moments = Lognormal(mean, std)
print(expected_moments.mu, expected_moments.sigma)

llm_moments = llm_lognormal(mean, std)
print(llm_moments)

llm_moments == (expected_moments.mu, expected_moments.sigma)

4.022977459259615 1.2924439729790718
(4.022977459259615, 1.2924439729790718)


True

## Sources reported by the LLM

No sources reported

## LLM output Stage 2

Here are the main changes and additions:
A critical_care_beds resource is created with a capacity of 24 beds using simpy.Resource(env, capacity=24).
A lognormal_to_normal function is added to convert the lognormal distribution parameters (mean and standard deviation) to the underlying normal distribution parameters (mu and sigma). This is necessary because SimPy's lognormal function expects the parameters of the underlying normal distribution.
The lognormal distribution parameters for each source are converted to the underlying normal distribution parameters using the lognormal_to_normal function in the __init__ method.
The unplanned_admissions_process method is added, which handles the admission, treatment, and discharge of patients. Here's what happens in this method:
The patient requests a critical care bed using self.critical_care_beds.request().
Once a bed is available, the patient is admitted, and the length of stay is determined based on the source-specific lognormal distribution.
The patient undergoes treatment for the determined length of stay using self.env.timeout(length_of_stay).
After treatment, the patient is discharged from the critical care bed.
Intensive cleaning takes place for 5 hours using self.env.timeout(5).
In the patient arrival generator functions (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals), a new process is started for the unplanned_admissions_process using self.env.process(self.unplanned_admissions_process(patient_id, source)).
The model prints useful information after each event, such as patient arrival, admission, discharge, and intensive cleaning completion.
With these changes, the model now includes the unplanned admissions treatment process, where patients request and wait for a critical care bed, undergo treatment based on their source-specific length of stay distribution, and then undergo intensive cleaning before the bed is released.

In [1]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Convert lognormal parameters to underlying normal distribution
        self.accident_emergency_mu, self.accident_emergency_sigma = self.lognormal_to_normal(128.79, 267.51)
        self.wards_mu, self.wards_sigma = self.lognormal_to_normal(177.89, 276.54)
        self.emergency_surgery_mu, self.emergency_surgery_sigma = self.lognormal_to_normal(140.15, 218.02)
        self.other_hospitals_mu, self.other_hospitals_sigma = self.lognormal_to_normal(212.86, 457.67)
        self.xray_department_mu, self.xray_department_sigma = self.lognormal_to_normal(87.53, 108.67)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            patient_id = self.patient_id_counter
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))
            self.patient_id_counter += 1

    def wards_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            patient_id = self.patient_id_counter
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))
            self.patient_id_counter += 1

    def emergency_surgery_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            patient_id = self.patient_id_counter
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))
            self.patient_id_counter += 1

    def other_hospitals_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            patient_id = self.patient_id_counter
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))
            self.patient_id_counter += 1

    def xray_department_arrivals(self):
        while True:
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            patient_id = self.patient_id_counter
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))
            self.patient_id_counter += 1

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                length_of_stay = self.rng_accident_emergency.lognormal(self.accident_emergency_mu, self.accident_emergency_sigma)
            elif source == "Wards":
                length_of_stay = self.rng_wards.lognormal(self.wards_mu, self.wards_sigma)
            elif source == "Emergency Surgery":
                length_of_stay = self.rng_emergency_surgery.lognormal(self.emergency_surgery_mu, self.emergency_surgery_sigma)
            elif source == "Other Hospitals":
                length_of_stay = self.rng_other_hospitals.lognormal(self.other_hospitals_mu, self.other_hospitals_sigma)
            else:  # X-Ray Department
                length_of_stay = self.rng_xray_department.lognormal(self.xray_department_mu, self.xray_department_sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 0 from Wards requests a critical care bed at 4.30 hours
Patient 0 from Wards admitted to a critical care bed at 4.30 hours
Patient 1 from Wards requests a critical care bed at 6.54 hours
Patient 1 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Patient 1 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 1 from Wards at 22.12 hours
Patient 3 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 3 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Patient 4 from Other Hospitals requests a critical care bed at 35.95 hours
Patient 4 from Other Hospitals admitted to a critical care bed at 35.95 hours
Patient 5 from Wards requests a critical care bed at 37.51 hours
Patient 5 from Wards admitted to a critical care bed at 37.51 hou

## Additional prompt
Modify the code so that in all generator functions, the patient_id_counter is increased before the arrival process

The changes made in this code are in the generator functions (accident_emergency_arrivals, wards_arrivals, emergency_surgery_arrivals, other_hospitals_arrivals, and xray_department_arrivals). In each of these functions, the patient_id_counter is incremented before the arrival process starts. The patient_id is assigned the current value of self.patient_id_counter, and then self.patient_id_counter is incremented by 1. After that, the arrival process is started with the appropriate inter-arrival time distribution, and the unplanned_admissions_process is started with the assigned patient_id.
This change ensures that the patient_id_counter is incremented before the arrival process starts, as per the requirement.

In [18]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Convert lognormal parameters to underlying normal distribution
        self.accident_emergency_mu, self.accident_emergency_sigma = self.lognormal_to_normal(128.79, 267.51)
        self.wards_mu, self.wards_sigma = self.lognormal_to_normal(177.89, 276.54)
        self.emergency_surgery_mu, self.emergency_surgery_sigma = self.lognormal_to_normal(140.15, 218.02)
        self.other_hospitals_mu, self.other_hospitals_sigma = self.lognormal_to_normal(212.86, 457.67)
        self.xray_department_mu, self.xray_department_sigma = self.lognormal_to_normal(87.53, 108.67)

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                length_of_stay = self.rng_accident_emergency.lognormal(self.accident_emergency_mu, self.accident_emergency_sigma)
            elif source == "Wards":
                length_of_stay = self.rng_wards.lognormal(self.wards_mu, self.wards_sigma)
            elif source == "Emergency Surgery":
                length_of_stay = self.rng_emergency_surgery.lognormal(self.emergency_surgery_mu, self.emergency_surgery_sigma)
            elif source == "Other Hospitals":
                length_of_stay = self.rng_other_hospitals.lognormal(self.other_hospitals_mu, self.other_hospitals_sigma)
            else:  # X-Ray Department
                length_of_stay = self.rng_xray_department.lognormal(self.xray_department_mu, self.xray_department_sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()


Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 5 from Wards requests a critical care bed at 6.54 hours
Patient 5 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Patient 5 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 5 from Wards at 22.12 hours
Patient 7 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 7 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Patient 3 from Other Hospitals requests a critical care bed at 35.95 hours
Patient 3 from Other Hospitals admitted to a critical care bed at 35.95 hours
Patient 6 from Wards requests a critical care bed at 37.51 hours
Patient 6 from Wards admitted to a critical care bed at 37.51 hou

## Additional prompt:

modify the code so that the lognormal parameters are stored separately before being used in the lognormal_to_normal function, and the lognormal_to_normal function is called in the unplanned admissions process rather than in the __init__ method.

In this modified code, the lognormal parameters (mu and sigma) for each source are stored separately in the __init__ method. These lognormal parameters are then used in the lognormal_to_normal function to convert them to the underlying normal distribution parameters (mu and sigma). The rest of the code remains the same, using the converted mu and sigma values for sampling the length of stay from the lognormal distribution.
This modification separates the storage of the lognormal parameters from their conversion to the underlying normal distribution parameters, making it easier to manage and update the lognormal parameters if needed. In this modified code, the lognormal_to_normal function is no longer called in the __init__ method. Instead, it is called within the unplanned_admissions_process method for each source. For each source, the lognormal_to_normal function is called with the corresponding lognormal parameters (mu and sigma) to convert them to the underlying normal distribution parameters. These converted mu and sigma values are then used to sample the length of stay from the lognormal distribution using the appropriate random number generator (rng_accident_emergency, rng_wards, rng_emergency_surgery, rng_other_hospitals, or rng_xray_department).
The rest of the code remains the same, including the storage of the lognormal parameters in the __init__ method.
This modification moves the conversion from lognormal to normal distribution parameters closer to where it is actually used (in the unplanned_admissions_process method), which can make the code more readable and easier to maintain.

In [23]:
import simpy
import numpy as np
from scipy.stats import lognorm

class CCU:
    def __init__(self, env, run_length=12 * 30 * 24):
        self.env = env
        self.run_length = run_length
        self.patient_id_counter = 0

        # Set up random number generators for each patient type
        self.rng_accident_emergency = np.random.default_rng(seed=42)
        self.rng_wards = np.random.default_rng(seed=43)
        self.rng_emergency_surgery = np.random.default_rng(seed=44)
        self.rng_other_hospitals = np.random.default_rng(seed=45)
        self.rng_xray_department = np.random.default_rng(seed=46)

        # Set up critical care bed resource
        self.critical_care_beds = simpy.Resource(env, capacity=24)

        # Lognormal parameters
        self.accident_emergency_lognormal_mu = 128.79
        self.accident_emergency_lognormal_sigma = 267.51
        self.wards_lognormal_mu = 177.89
        self.wards_lognormal_sigma = 276.54
        self.emergency_surgery_lognormal_mu = 140.15
        self.emergency_surgery_lognormal_sigma = 218.02
        self.other_hospitals_lognormal_mu = 212.86
        self.other_hospitals_lognormal_sigma = 457.67
        self.xray_department_lognormal_mu = 87.53
        self.xray_department_lognormal_sigma = 108.67

    def lognormal_to_normal(self, mu, sigma):
        """Convert lognormal parameters to underlying normal distribution"""
        zeta = np.log(mu**2 / np.sqrt(sigma**2 + mu**2))
        sigma_norm = np.sqrt(np.log(sigma**2 / mu**2 + 1))
        mu_norm = zeta
        return mu_norm, sigma_norm

    def accident_emergency_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_accident_emergency.exponential(22.72))
            self.env.process(self.unplanned_admissions_process(patient_id, "Accident and Emergency"))

    def wards_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_wards.exponential(26.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Wards"))

    def emergency_surgery_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_emergency_surgery.exponential(37.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "Emergency Surgery"))

    def other_hospitals_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_other_hospitals.exponential(47.2))
            self.env.process(self.unplanned_admissions_process(patient_id, "Other Hospitals"))

    def xray_department_arrivals(self):
        while True:
            patient_id = self.patient_id_counter
            self.patient_id_counter += 1
            yield self.env.timeout(self.rng_xray_department.exponential(575.0))
            self.env.process(self.unplanned_admissions_process(patient_id, "X-Ray Department"))

    def unplanned_admissions_process(self, patient_id, source):
        print(f"Patient {patient_id} from {source} requests a critical care bed at {self.env.now:.2f} hours")
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {patient_id} from {source} admitted to a critical care bed at {self.env.now:.2f} hours")

            # Determine length of stay based on source
            if source == "Accident and Emergency":
                mu, sigma = self.lognormal_to_normal(self.accident_emergency_lognormal_mu, self.accident_emergency_lognormal_sigma)
                length_of_stay = self.rng_accident_emergency.lognormal(mu, sigma)
            elif source == "Wards":
                mu, sigma = self.lognormal_to_normal(self.wards_lognormal_mu, self.wards_lognormal_sigma)
                length_of_stay = self.rng_wards.lognormal(mu, sigma)
            elif source == "Emergency Surgery":
                mu, sigma = self.lognormal_to_normal(self.emergency_surgery_lognormal_mu, self.emergency_surgery_lognormal_sigma)
                length_of_stay = self.rng_emergency_surgery.lognormal(mu, sigma)
            elif source == "Other Hospitals":
                mu, sigma = self.lognormal_to_normal(self.other_hospitals_lognormal_mu, self.other_hospitals_lognormal_sigma)
                length_of_stay = self.rng_other_hospitals.lognormal(mu, sigma)
            else:  # X-Ray Department
                mu, sigma = self.lognormal_to_normal(self.xray_department_lognormal_mu, self.xray_department_lognormal_sigma)
                length_of_stay = self.rng_xray_department.lognormal(mu, sigma)

            yield self.env.timeout(length_of_stay)
            print(f"Patient {patient_id} from {source} discharged from a critical care bed at {self.env.now:.2f} hours")

            # Intensive cleaning
            yield self.env.timeout(5)
            print(f"Intensive cleaning completed for Patient {patient_id} from {source} at {self.env.now:.2f} hours")

    def run(self):
        # Start patient arrival processes
        accident_emergency_process = self.env.process(self.accident_emergency_arrivals())
        wards_process = self.env.process(self.wards_arrivals())
        emergency_surgery_process = self.env.process(self.emergency_surgery_arrivals())
        other_hospitals_process = self.env.process(self.other_hospitals_arrivals())
        xray_department_process = self.env.process(self.xray_department_arrivals())

        self.env.run(until=self.run_length)

# Run the simulation
env = simpy.Environment()
ccu = CCU(env)
ccu.run()



Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 5 from Wards requests a critical care bed at 6.54 hours
Patient 5 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Patient 5 from Wards discharged from a critical care bed at 17.12 hours
Intensive cleaning completed for Patient 5 from Wards at 22.12 hours
Patient 7 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 7 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Patient 3 from Other Hospitals requests a critical care bed at 35.95 hours
Patient 3 from Other Hospitals admitted to a critical care bed at 35.95 hours
Patient 6 from Wards requests a critical care bed at 37.51 hours
Patient 6 from Wards admitted to a critical care bed at 37.51 hou

## Testing

* Extreme value test 1: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result (PASS): queues form after first arrival. 
* Extreme value test 2: means of arrival_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result (PASS): after 24 arrivals queues form and no patients are admitted.

### Extreme value test 1

PASS.

In [19]:
env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu = CCU(env)
ccu.critical_care_beds = simpy.Resource(env, capacity=1) 
ccu.run()


Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 5 from Wards requests a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 7 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 3 from Other Hospitals requests a critical care bed at 35.95 hours
Patient 6 from Wards requests a critical care bed at 37.51 hours
Patient 1 from Wards discharged from a critical care bed at 54.59 hours
Patient 0 from Accident and Emergency requests a critical care bed at 54.62 hours
Intensive cleaning completed for Patient 1 from Wards at 59.59 hours
Patient 5 from Wards admitted to a critical care bed at 59.59 hours
Patient 8 from Emergency Surgery requests a critical care bed at 76.27 hours
Patient 9 from Other Hospitals requests a critical care bed at 77.00 hours
Patient 10 from Wards requests a critical care bed at 77.91 hours
Pa

## Extreme value test 2

PASS

In [25]:
M = 10_000_000

env = simpy.Environment()

# Create an instance of the CCU model and run the simulation
ccu = CCU(env)

ccu.accident_emergency_lognormal_mu = M
ccu.accident_emergency_lognormal_sigma = 267.51
ccu.wards_lognormal_mu = M
ccu.wards_lognormal_sigma = 276.54
ccu.emergency_surgery_lognormal_mu = M
ccu.emergency_surgery_lognormal_sigma = 218.02
ccu.other_hospitals_lognormal_mu = M
ccu.other_hospitals_lognormal_sigma = 457.67
ccu.xray_department_lognormal_mu = M
ccu.xray_department_lognormal_sigma = 108.67

ccu.run()

Patient 1 from Wards requests a critical care bed at 4.30 hours
Patient 1 from Wards admitted to a critical care bed at 4.30 hours
Patient 5 from Wards requests a critical care bed at 6.54 hours
Patient 5 from Wards admitted to a critical care bed at 6.54 hours
Patient 2 from Emergency Surgery requests a critical care bed at 13.71 hours
Patient 2 from Emergency Surgery admitted to a critical care bed at 13.71 hours
Patient 7 from Emergency Surgery requests a critical care bed at 31.66 hours
Patient 7 from Emergency Surgery admitted to a critical care bed at 31.66 hours
Patient 3 from Other Hospitals requests a critical care bed at 35.95 hours
Patient 3 from Other Hospitals admitted to a critical care bed at 35.95 hours
Patient 6 from Wards requests a critical care bed at 37.51 hours
Patient 6 from Wards admitted to a critical care bed at 37.51 hours
Patient 10 from Wards requests a critical care bed at 48.02 hours
Patient 10 from Wards admitted to a critical care bed at 48.02 hours
Pat